# 编译配套工具

之前的代码我们直接使用gcc来手工编译,这没问题,不过我们也看到了每次编译的命令都比python的helloworld都长了.像很多复杂的程序用到很多外部依赖的情况下编译都快赶上写业务逻辑了.

在js中我们用glup来流程化编译,在c中的流程化编译工具就太多了,各自适用于不同的平台有着不同的规范,这边介绍的是[makefile](https://www.gnu.org/software/make/manual/make.html)和[cmake](https://cmake.org/)工具.

makefile是类unix系统下一个古老的工具,至今活跃在编程届.

cmake是一个高层次的流程化编译工具,它可以用来自动生成makefile,关键是他跨平台.

## makefile与自动化编译

make是一个linux/osx下可以替你运行编译命令的工具.make会检查源文件和目标文件的时间戳,如果目标文件过期,make就会重新编译它.

在windows下,只有安装了mingw才会安装make工具,它的名字叫做`mingw32-make.exe`.为了方便起见可以将其改名为`make`

但是做到所有这些事情前,需要告诉make源代码的一些情况.make需要知道文件之间的依赖关系,同时还需要告诉它你具体想如何构建代码.严格意义上讲,make不仅仅可以用来编译文件.目标可以是任何用其他文件生成的文件,也就是说目标可以是一批文件压缩而成的压缩文档.

对每个目标,make需要知道两件事:

+ 依赖项

    生成目标需要用哪些文件
    
    
+ 生成方法

    生成该文件时要用哪些指令
    
    
依赖项和生成方法合在一起构成了一条规则.有了规则,make就知道如何生成目标.

## 用makefile向make描述代码

所有目标,依赖项和生成方法的细节信息需要保存在一个叫`makefile`或`Makefile`的文件中,

### makefile语法

makefile规则很简单,基本的规则:

+ 规则形式

```makefile
target: require1 require2 require3
	process
```
注意process前面必须是tab

+ 定义环境变量

```makefile
export FLASK_ENV=dev
```
+ 定义变量

```makefile
DIR=xxx
```

+ 使用环境变量/变量

```makefile
$(pwd)
```

注意,要获取标准输出的内容需要使用`$(shell xxx)`

为了弄明白它是怎么工作的,下面我们还是以`binary_vector`为例为它写个`makefile`用以实现编译静态链接库和测试文件的功能:

In [1]:
%%writefile ./code/compiler/S2/makefile
DIR = $(shell pwd)
INCLUDE_DIR = $(DIR)/source/inc
SRC_DIR = $(DIR)/source/src
OBJECT_DIR = $(DIR)/source/obj
LIB_DIR = $(DIR)/build/lib
BIN_DIR = $(DIR)/build/bin
TEST_DIR = $(DIR)/test

objects = $(addprefix $(OBJECT_DIR)/,binary_operator.o unary_operator.o struct_operator.o)

all: $(LIB_DIR)/libvector.a
	echo "done!"

$(OBJECT_DIR)/struct_operator.o: $(INCLUDE_DIR)/binary_vector.h
	gcc -c \
-o $(OBJECT_DIR)/struct_operator.o \
-I $(INCLUDE_DIR) \
$(SRC_DIR)/struct_operator.c    

$(OBJECT_DIR)/binary_operator.o: $(INCLUDE_DIR)/binary_vector.h
	gcc -c \
-o $(OBJECT_DIR)/binary_operator.o \
-I $(INCLUDE_DIR) \
$(SRC_DIR)/binary_operator.c
    
$(OBJECT_DIR)/unary_operator.o: $(INCLUDE_DIR)/binary_vector.h
	gcc -c \
-o $(OBJECT_DIR)/unary_operator.o \
-I $(INCLUDE_DIR) \
$(SRC_DIR)/unary_operator.c
    
$(LIB_DIR)/libvector.a: $(objects)
	ar crv $(LIB_DIR)/libvector.a \
$(objects)

$(BIN_DIR)/vectorTest: $(LIB_DIR)/libvector.a $(INCLUDE_DIR)/binary_vector.h $(TEST_DIR)/test.c
	gcc \
-o $(BIN_DIR)/vectorTest \
-I $(INCLUDE_DIR) \
-L $(LIB_DIR) \
-l vector \
$(TEST_DIR)/test.c

test: $(BIN_DIR)/vectorTest
	$(BIN_DIR)/vectorTest
    
clean: $(BIN_DIR)/vectorTest $(objects) $(LIB_DIR)/libvector.a
	rm $(BIN_DIR)/vectorTest $(objects) $(LIB_DIR)/libvector.a

Overwriting ./code/compiler/S2/makefile


需要注意的是每条命令部分只能用`/tab`而不是空格

### 使用`make`命令

make命令使用`-C`指定操作目录, 后面接`makefile`中的target执行定义的过程和依赖.如果不指定target则会知名第一条target

In [2]:
!make -C ./code/compiler/S2/ clean

gcc \
-o /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/build/bin/vectorTest \
-I /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/inc \
-L /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/build/lib \
-l vector \
/Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/test/test.c
rm /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/build/bin/vectorTest /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/obj/binary_operator.o /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/obj/unary_operator.o /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/obj/struct_operator.o /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/build/lib/libvector.a


In [3]:
!make -C ./code/compiler/S2/

gcc -c \
-o /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/obj/binary_operator.o \
-I /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/inc \
/Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/src/binary_operator.c
gcc -c \
-o /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/obj/unary_operator.o \
-I /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/inc \
/Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/src/unary_operator.c
gcc -c \
-o /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/obj/struct_operator.o \
-I /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/inc \
/Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/so

In [4]:
!make -C ./code/compiler/S2/ test

gcc \
-o /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/build/bin/vectorTest \
-I /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/source/inc \
-L /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/build/lib \
-l vector \
/Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/test/test.c
/Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S2/build/bin/vectorTest
mod(A)=2.236068
mod(B)=5.000000
mul(A,B)=11.000000
add(A,B)=<4.000000,6.000000>


## 使用cmake流程化编译

Cmake本质上也不提供流程化编译的功能,他其实是用来生成不同平台make文件的,在linux,mac osx上就是`makefile`,正真编译使用`make`命令,在windows上就是`nmake`当然windows下也可以使用`mingw`来编译,自带的`mingwXX-make`来编译makefile.不过这需要在cmake的时候加上额外参数`-G  "MinGW Makefiles"`指定

### Cmake基本用法

Cmake是指令式的配置方式,基本形式就是`cmake_minimum_required (VERSION 2.8)`这样,`()`外面的是参数字段,里面的是内容.

#### 常用的字段有:

> 元信息设置

+ `cmake_minimum_required`：指定运行此配置文件所需的 CMake 的最低版本

+ `project(<proname>)`：参数值是 Demo1，该命令指定项目的名称

> 编译器设置

+ `set(CMAKE_ARCHIVE_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/Lib)`:设置静态连接文件输出目录
+ `set(CMAKE_LIBRARY_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/Lib)`:设置动态态连接文件输出目录
+ `set(CMAKE_RUNTIME_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/Bin)`:设置可执行文件输出目录
+ `set_target_properties(<projectnam> PROPERTIES PUBLIC_HEADER "some.h;headers.h")` 将对外的接口头文件设置到编译目标上
+ `include_directories([AFTER|BEFORE] [SYSTEM] dir1 [dir2 ...])`:设置头文件目录
+ `link_directories(directory1 directory2 ...)`:设置连接库所在位置
+ `link_libraries(library1 <debug | optimized> library2 ...)`:添加需要链接的库文件路径
+ `target_link_libraries(<target> [item1 [item2 [...]]][[debug|optimized|general] <item>] ...)`:设置要链接的库文件的名称
+ `aux_source_directory(<dir> <variable>)`: 指定源文件的所在目录,如`aux_source_directory(. DIR_SRCS)`这样就将所有文件夹下的文件名放到了`DIR_SRCS`变量中.`add_executable(Demo ${DIR_SRCS})`就指定好了所有文件.

+ `add_executable(<target> <sourcefiles...>)`： 将源文件编译成一个可执行文件,第一位是目标可执行文件名,后面则是源文件名.

+ add_compile_options(-std=c++11):指定编译目标使用的额外参数,比如支持c++11

+ target_compile_features(<target> <features...>)指定编译目标使用的额外参数

> 链接编译设置

+ `add_library (<target> [type]<sourcefiles...>)`:将源文件编译成一个链接库.


> 子目录设置

+ `add_subdirectory(math)`:添加子目录,子目录中需要有`CMakeLists.txt`文件,编译时会优先编译子目录.

> 测试设置

可以用`make test`执行测试

+ `enable_testing()`:启动测试
+ `add_test(<测试名> <test_target> <args>)`:添加一个测试,需要先把测试代码编译了
+ `set_tests_properties (test_usage PROPERTIES PASS_REGULAR_EXPRESSION "Usage: .* base exponent")`设定测试的属性


+ 定义一个宏，用来简化测试工作

```cmake
macro (do_test arg1 arg2 result)
  add_test (test_${arg1}_${arg2} Demo ${arg1} ${arg2})
  set_tests_properties (test_${arg1}_${arg2}
    PROPERTIES PASS_REGULAR_EXPRESSION ${result})
endmacro (do_test)
```

> debug测试

```cmake
set(CMAKE_BUILD_TYPE "Debug")
set(CMAKE_CXX_FLAGS_DEBUG "$ENV{CXXFLAGS} -O0 -Wall -g -ggdb")
set(CMAKE_CXX_FLAGS_RELEASE "$ENV{CXXFLAGS} -O3 -Wall")
```

> install功能

通常可以是这样的定义

```cmake
install([TARGETS <name>...(编译目标)|FILES <file>...(要移动的文件)]
    [LIBRARY(动态库)|ARCHIVE(静态库)|RUNTIME(可执行文件)] DESTINATION  <dir> ...
    [INCLUDES DESTINATION [<dir> ...]]
)  
```

#### cmake 的语法有:

+ 打印信息

    `MESSAGE([SEND_ERROR | STATUS | FATAL_ERROR] "message to display"...)`,如:`MESSAGE(STATUS "This is BINARY dir " ${HELLO_BINARY_DIR}))`

+ 指定编译器:

    `SET(CMAKE_C_COMPILER "/home/public/local/bin/gcc")`

    `SET(CMAKE_CXX_COMPILER "/home/public/local/bin/g++")`

+ 显式的设置自定义变量

    使用`set(变量名 值)`的操作，如:`SET(HELLO_SRC main.SOURCE_PATHc)`
    
+ 变量引用的方式

    使用`${}`进行变量的引用
    
+ 设置环境变量

    `SET(ENV{变量名} 值)`
    
+ 调用环境变量

    使用`$ENV{NAME}`指令就可以调用系统的环境变量了
    
    
+ IF 指令

   基本语法为:
   ```
   IF(expression_r_r)
     # THEN section.
     COMMAND1(ARGS ...)
     COMMAND2(ARGS ...)
     ...
   ELSE(expression_r_r)
     # ELSE section.
     COMMAND1(ARGS ...)
     COMMAND2(ARGS ...)
     ...
   ENDIF(expression_r_r)
   ```
   
+ WHILE指令

   基本语法为:
   ```
    WHILE(condition)
      COMMAND1(ARGS ...)
      COMMAND2(ARGS ...)
      ...
    ENDWHILE(condition)
   ```
其真假判断条件可以参考 IF 指令。
    
    
+ FOREACH

    FOREACH 指令的使用方法有两种形式:
    1. 列表
    ```
    FOREACH(loop_var arg1 arg2 ...)
      COMMAND1(ARGS ...)
      COMMAND2(ARGS ...)
      ...
    ENDFOREACH(loop_var)
    ```
    2. 范围(类似pythonz中的for xxx in range(x))
    ```
    FOREACH(loop_var RANGE oop_var RANGE start stop [step])
    ENDFOREACH(loop_var)
    ```

#### Cmake的预定义变量有:

+ `CMAKE_BINARY_DIR/PROJECT_BINARY_DIR/<projectname>_BINARY_DIR`

    这三个变量指代的内容是一致的,如果是`in source`编译,指得就是工程顶层目录,如果是`out-of-source`编译,指的是工程编译发生的目录
    PROJECT_BINARY_DIR 跟其他指令稍有区别,但多数时候你可以理解为他们是一致的.
    
    
+ `CMAKE_SOURCE_DIR/PROJECT_SOURCE_DIR/<projectname>_SOURCE_DIR`

    这三个变量指代的内容是一致的,不论采用何种编译方式,都是工程顶层目录.也就是在`in source`编译时,他跟`CMAKE_BINARY_DIR`等变量一致.`PROJECT_SOURCE_DIR`跟其他指令稍有区别,但多数时候你可以理解为他们是一致的.

+ `CMAKE_CURRENT_SOURCE_DIR`

    指的是当前处理的`CMakeLists.txt`所在的路径

+ `CMAKE_CURRRENT_BINARY_DIR`

    如果是`in-source`编译,它跟`CMAKE_CURRENT_SOURCE_DIR`一致,如果是`out-of-source`编译,他指的是`target`编译目录.使用`ADD_SUBDIRECTORY(src bin)`可以更改这个变量的值.使用`SET(EXECUTABLE_OUTPUT_PATH <新路径>)`并不会对这个变量造成影响,它仅仅修改了最终目标文件存放的路径.

+ `CMAKE_CURRENT_LIST_FILE`

    输出调用这个变量的`CMakeLists.txt`的完整路径

 
+ `CMAKE_CURRENT_LIST_LINE`

    输出这个变量所在的行

+ CMAKE_MODULE_PATH

    这个变量用来定义自己的`cmake`模块所在的路径.如果你的工程比较复杂,有可能会自己编写一些`cmake`模块,这些`cmake`模块是随你的工程发布的,为了让 `cmake`在处理`CMakeLists.txt`时找到这些模块,你需要通过`SET`指令,将自己的`cmake`模块路径设置一下.比如`SET(CMAKE_MODULE_PATH ${PROJECT_SOURCE_DIR}/cmake)`.这时候你就可以通过 INCLUDE 指令来调用自己的模块了.


+ `EXECUTABLE_OUTPUT_PATH /LIBRARY_OUTPUT_PATH`

    分别用来重新定义最终结果的存放目录,前面我们已经提到了这两个变量。

+ `PROJECT_NAME`

    返回通过`PROJECT`指令定义的项目名称.


+ `CMAKE_MAJOR_VERSION,CMAKE`

    主版本号,比如 2.4.6 中的 2
    
    
+ `CMAKE_MINOR_VERSION,CMAKE`

    次版本号,比如 2.4.6 中的 4
    
    
+ `CMAKE_PATCH_VERSION,CMAKE`

    补丁等级,比如 2.4.6 中的 6
    

+ `CMAKE_SYSTEM`

    系统名称,比如 Linux-2.6.22
    
    
+ `CMAKE_SYSTEM_NAME`

    不包含版本的系统名,比如 Linux
    
    
+ `CMAKE_SYSTEM_VERSION`

    系统版本,比如 2.6.22
    
    
+ `CMAKE_SYSTEM_PROCESSOR`

    处理器名称,比如 i686.
    
    
+ `UNIX`

    在所有的类 UNIX 平台为 TRUE,包括 OS X 和 cygwin
    
+ `WIN32`

    在所有的 win32 平台为 TRUE,包括 cygwin
    
+ `CMAKE_INSTALL_PREFIX`

    在cmake生成makefile时使用`-D`指定的目录,用于指定安装目录,如果不指定且安装的地址是相对地址,就会默认安装到`/usr/local/`下.

### 使用cmake编译我们的二元向量

更多的用法可以看官方文档.本文不会叙述太多.针对我们的二元向量模块,我们需要写两个`CMakeLists.txt`,一个用于将src中的内容编译为静态库,一个用于编译测试文件.

我们先将S2中的代码复制到S3

In [5]:
%%writefile ./code/compiler/S3/CMakeLists.txt
#项目编译环境
cmake_minimum_required (VERSION 2.8)
project (binary_vector)
include_directories(source/inc)
# 编译动态链接库
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/build/lib)
aux_source_directory(${CMAKE_BINARY_DIR}/source/src DIR_LIB_SRCS)
add_library(vector SHARED ${DIR_LIB_SRCS})
install(TARGETS   vector
    LIBRARY DESTINATION  CMAKE_LIBRARY_OUTPUT_DIRECTORY  
)  
# 编译连接生成demo
set(CMAKE_RUNTIME_OUTPUT_DIRECTORY ${CMAKE_BINARY_DIR}/build/bin)
link_directories(${CMAKE_BINARY_DIR}/build/lib)
link_libraries(vector)
aux_source_directory(test DIR_SRCS)
add_executable(Demo ${DIR_SRCS})
#测试
enable_testing()
add_test (DemoRuns ${CMAKE_BINARY_DIR}/build/bin/Demo)

Overwriting ./code/compiler/S3/CMakeLists.txt


In [6]:
!cmake ./code/compiler/S3/CMakeLists.txt

-- Configuring done
CMake Warning (dev):
  Policy CMP0042 is not set: MACOSX_RPATH is enabled by default.  Run "cmake
  --help-policy CMP0042" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

  MACOSX_RPATH is not specified for the following targets:

   vector

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Generating done
-- Build files have been written to: /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S3


cmake会为我们创建一个更好的makefile

In [8]:
!make -C ./code/compiler/S3/

[ 66%] Built target vector
[ 83%] Building C object CMakeFiles/Demo.dir/test/test.c.o
[100%] Linking C executable build/bin/Demo
[100%] Built target Demo


In [9]:
!make -C ./code/compiler/S3/ test

Running tests...
Test project /Users/huangsizhe/Workspace/Documents/TutorialForCLang/c-tool-chain/code/compiler/S3
    Start 1: DemoRuns
1/1 Test #1: DemoRuns .........................   Passed    0.01 sec

100% tests passed, 0 tests failed out of 1

Total Test time (real) =   0.02 sec
